<a href="https://colab.research.google.com/github/GuyAvni1/Final-Project-Guy-Ori/blob/main/Preproccesing/Preparation_for_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparation for Prediction**
In this part we prepare the data for prediction.
Each patient get it's label from the present to the future with a timelapse of 'month' months (changeable parameter).

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Reading the data set from the drive
df = pd.read_csv("/content/drive/MyDrive/FPD/df_imputed.csv")

In [4]:
import pandas as pd
from google.colab import files

# Function to convert VISCODE to months
def viscode_to_month(viscode):
    return int(viscode[1:]) if viscode.startswith('m') else 0

# Replace VISCODE with integer values
df['VISCODE'] = df['VISCODE'].apply(viscode_to_month)

# Create a dictionary to store the next appointment month and DX for each patient
next_appointment_info = {}

# Add a new column 'Transition' to store the transition label
df['Transition'] = ''

# Loop through the DataFrame to find next appointments and set the transition label
month = 12 # Number of month for prediction. Change as you wish.

for index, row in df.iterrows():
    patient_id = row['RID']
    current_month = row['VISCODE']
    next_month = current_month + month

    # Check if the next appointment exists for the patient
    next_appointment_row = df[(df['RID'] == patient_id) & (df['VISCODE'] == next_month)]
    if not next_appointment_row.empty:
        next_appointment_info[index] = {
            'next_month': next_month,
            'next_DX': next_appointment_row['DX'].values[0]
        }

        current_DX = row['DX']
        next_DX = next_appointment_row['DX'].values[0]

        # Define transition labels
        if current_DX == 'CN' and next_DX == 'CN':
            transition_label = 'CNtoCN'
        elif current_DX == 'CN' and next_DX == 'MCI':
            transition_label = 'CNtoMCI'
        elif current_DX == 'MCI' and next_DX == 'MCI':
            transition_label = 'MCItoMCI'
        elif current_DX == 'MCI' and next_DX == 'Dementia':
            transition_label = 'MCItoAD'
        elif current_DX == 'Dementia' and next_DX == 'Dementia':
            transition_label = 'ADtoAD'
        else:
            transition_label = 'CNtoAD'

        # Set the transition label in the DataFrame
        df.at[index, 'Transition'] = transition_label

# Filter rows that have a future meeting
filtered_df = df[df.index.isin(next_appointment_info.keys())]

# Save the filtered DataFrame to a new Excel file
filtered_excel_file = 'future_of_' + str(month) + 'm.csv'
filtered_df.to_csv(filtered_excel_file, index=False)
files.download(filtered_excel_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>